In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import re

In [2]:
newsData = pd.read_csv('Combined_News_DJIA.csv')
#remove all non-alphabetic char from headlines.
df = newsData.copy()
df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
for i in df.columns:
    if i=='Date':
        continue
    if i=='Label':
        continue
    df[i] = df[i].str.lower()
df['Date'] = newsData['Date']

In [3]:
df.replace("^b ","",regex=True, inplace=True)
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,georgia downs two russian warplanes as count...,breaking musharraf to be impeached,russia today columns of troops roll into sout...,russian tanks are moving towards the capital o...,afghan children raped with impunity u n of...,russian tanks have entered south ossetia w...,breaking georgia invades south ossetia russi...,the enemy combatent trials are nothing but a...,georgian troops retreat from s osettain capit...,did the u s prep georgia for war with russia,rice gives green light for israel to attack ir...,announcing class action lawsuit on behalf of a...,so russia and georgia are at war and the nyt...,china tells bush to stay out of other countrie...,did world war iii start today,georgia invades south ossetia if russia gets...,al qaeda faces islamist backlash,condoleezza rice the us would not act to pre...,this is a busy day the european union has ap...,georgia will withdraw soldiers from iraq...,why the pentagon thinks attacking iran is a ba...,caucasus in crisis georgia invades south osse...,indian shoe manufactory and again in a seri...,visitors suffering from mental illnesses banne...,no help for mexico s kidnapping surge
1,2008-08-11,1,why wont america and nato help us if they won...,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli tr...,georgian army flees in disarray as russians ad...,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zeala...,russia angered by israeli military sale to geo...,an american citizen living in s ossetia blames...,welcome to world war iv now in high definition,georgia s move a mistake of monumental propor...,russia presses deeper into georgia u s says ...,abhinav bindra wins first ever individual olym...,u s ship heads for arctic to define territory,drivers in a jerusalem taxi station threaten t...,the french team is stunned by phelps and the ...,israel and the us behind the georgian aggressi...,do not believe tv neither russian nor georgi...,riots are still going on in montreal canada ...,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over...,perhaps the question about the georgia rus...,russia is so much better at war,so this is what it s come to trading sex for ...
2,2008-08-12,0,remember that adorable year old who sang at ...,russia ends georgia operation,if we had no sexual harassment we would have ...,al qa eda is losing support in iraq because of...,ceasefire in georgia putin outmaneuvers the w...,why microsoft and intel tried to kill the xo ...,stratfor the russo georgian war and the balan...,i m trying to get a sense of this whole georgi...,the us military was surprised by the timing an...,u s beats war drum as iran dumps the dollar,gorbachev georgian military attacked the sou...,cnn use footage of tskhinvali ruins to cover g...,beginning a war as the olympics were opening v...,pyramids as large as the luxor stacked into...,the top party cities in the world,u s troops still in georgia did you know the...,why russias response to georgia was right,gorbachev accuses u s of making a serious bl...,russia georgia and nato cold war two,remember that adorable year old who led you...,war in georgia the israeli connection,all signs point to the us encouraging georgia ...,christopher king argues that the us and nato a...,america the new mexico,bbc news asia pacific extinction by man n...
3,2008-08-13,0,u s refuses israel weapons to attack iran r...,when the president ordered to attack tskhinval...,israel clears troops who killed reuters camer...,britain s policy of being tough on drugs is ...,body of year old found in trunk latest ra...,china has moved million quake survivors i...,bush announces operation get all up in russia ...,russian forces sink g

In [4]:
#remove NA and replace with empty string
df.fillna(' ', inplace=True)

In [5]:
def mergeCol(row):
    return ' '.join(row['Top1':'Top25'])
df['headlines'] = df.apply(mergeCol, axis = 1)

In [6]:
new_df = df[['Label','headlines']]

In [32]:
vocabSize = 1500
tokenizer = Tokenizer(num_words=vocabSize, split=' ')
tokenizer.fit_on_texts(new_df['headlines'].values)
X = tokenizer.texts_to_sequences(new_df['headlines'])
X = pad_sequences(X)

In [33]:
X.shape

(1989, 553)

In [42]:
embed_dim = 128

lstm_out = 300

model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.1, recurrent_dropout=0.1))

In [43]:
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [44]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()
y = Le.fit_transform(new_df['Label'])

In [45]:
train = df['Date'] < '20150101'
test = df['Date'] > '20141231'

In [46]:
X_train = X[train,:]
X_test = X[test,:]
y_train = y[train]
y_test = y[test]

In [47]:
model.fit(X_train, y_train,validation_data = (X_test,y_test),epochs = 10, batch_size=32)

Epoch 1/10
59/59 [==============================] - 214s 4s/step - loss: 0.6922 - accuracy: 0.5393 - val_loss: 0.6857 - val_accuracy: 0.5529
Epoch 2/10
59/59 [==============================] - 215s 4s/step - loss: 0.6896 - accuracy: 0.6063 - val_loss: 0.5729 - val_accuracy: 0.7434
Epoch 3/10
59/59 [==============================] - 216s 4s/step - loss: 0.5071 - accuracy: 0.7742 - val_loss: 0.5069 - val_accuracy: 0.7672
Epoch 4/10
59/59 [==============================] - 216s 4s/step - loss: 0.3229 - accuracy: 0.8739 - val_loss: 0.4413 - val_accuracy: 0.8042
Epoch 5/10
59/59 [==============================] - 215s 4s/step - loss: 0.2036 - accuracy: 0.9361 - val_loss: 0.4603 - val_accuracy: 0.8413
Epoch 6/10
59/59 [==============================] - 216s 4s/step - loss: 0.1204 - accuracy: 0.9640 - val_loss: 0.5025 - val_accuracy: 0.8545
Epoch 7/10
59/59 [==============================] - 215s 4s/step - loss: 0.0621 - accuracy: 0.9846 - val_loss: 0.5827 - val_accuracy: 0.8439
Epoch 8/10
59

In [48]:
model.evaluate(X_test,y_test)

12/12 [==============================] - 6s 475ms/step - loss: 0.8188 - accuracy: 0.8545


[0.8187647461891174, 0.8544973731040955]

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 553, 128)          192000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 300)               514800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 707,101
Trainable params: 707,101
Non-trainable params: 0
_________________________________________________________________
